# 非定常データの粗視化による予測精度の改善

このノートブックでは、非定常データに対する予測が困難であることを示し、その予測精度を向上させるために移動平均平滑化を用いる手法について説明する。
まず、非定常データを生成し、ARIMAモデルを用いてその予測を行う。
次に、異なるウィンドウサイズで移動平均を適用し、平滑化されたデータに対して同様にARIMAモデルを適用して予測を行う。
これにより、データの平滑化が予測精度向上に与える影響を確認する。

## 具体的な手順

1. **非定常データの生成**:
    - ランダムウォーク(酔歩)による非定常データを生成する。
    - このデータは、急激な変動を含んでおり、予測が困難な性質を持つ。

2. **ARIMAモデルによる予測**:
    - 非定常データのトレーニングセットに対してARIMAモデルを適用し、モデルをフィットさせる。
    - テストセットに対して予測を行い、その予測結果をプロットする。
    - このプロットにより、非定常データの予測の不確かさを視覚的に示す。

3. **移動平均平滑化の適用**:
    - 異なるウィンドウサイズ（10、40、80）で移動平均を適用し、データを平滑化する。
    - 平滑化されたデータをトレーニングセットとテストセットに分割する。

4. **平滑化データに対するARIMAモデルの適用**:
    - 平滑化されたトレーニングセットに対してARIMAモデルを適用し、モデルをフィットさせる。
    - テストセットに対して予測を行い、その予測結果をプロットする。
    - 各ウィンドウサイズでの予測結果を比較し、平滑化による予測精度向上の効果を確認する。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# 非定常データの生成
np.random.seed(0)
time = np.arange(400)
#non_stationary_data = np.random.randn(400).cumsum() + np.sin(np.linspace(0, 20, 400))
non_stationary_data = np.random.randn(400).cumsum() 

# DataFrameに変換
df = pd.DataFrame({'value': non_stationary_data}, index=pd.date_range(start='2023-01-01', periods=400))

# トレーニングデータとテストデータに分割
train = df.iloc[:300]
test = df.iloc[300:]

# 非定常データに対するARIMAモデルの適用
model = ARIMA(train, order=(2, 1, 2))
results = model.fit()

# 予測
forecast = results.get_forecast(steps=100)
forecast_mean = forecast.predicted_mean
#forecast_conf_int = forecast.conf_int(alpha=0.05)

# プロット範囲の設定
x_limits = [df.index[0], df.index[-1]]

# 非定常データの予測プロット
plt.figure(figsize=(14, 3))
plt.plot(df, label='Original Data', color='blue')
plt.plot(results.fittedvalues, color='red', label='Fitted Values (Train Data)')
plt.plot(test.index, forecast_mean, color='green', label='Forecast')
#plt.fill_between(test.index, forecast_conf_int.iloc[:, 0], forecast_conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.xlim(x_limits)
plt.ylim(-5, 30)
plt.title('ARIMA Model on Non-Stationary Data: Forecasting Challenge')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

# 移動平均のウィンドウサイズを変えて比較
window_sizes = [10, 40, 80]
for window_size in window_sizes:
    # 移動平均の適用（粗視化）
    df_smoothed = df.rolling(window=window_size).mean()

    # トレーニングデータとテストデータに分割
    train_smoothed = df_smoothed.iloc[:300].dropna()
    test_smoothed = df_smoothed.iloc[300:]

    # 平滑化データに対するARIMAモデルの適用
    model_smoothed = ARIMA(train_smoothed, order=(2, 1, 2))
    results_smoothed = model_smoothed.fit()

    # 平滑化データの予測
    forecast_smoothed = results_smoothed.get_forecast(steps=100)
    forecast_mean_smoothed = forecast_smoothed.predicted_mean
    #forecast_conf_int_smoothed = forecast_smoothed.conf_int(alpha=0.05)

    # 平滑化データの予測プロット
    plt.figure(figsize=(14, 3))
    plt.plot(df_smoothed, label='Smoothed Data', color='blue')
    plt.plot(results_smoothed.fittedvalues, color='red', label='Fitted Values (Train Data)')
    plt.plot(test_smoothed.index, forecast_mean_smoothed, color='green', label='Forecast')
    #plt.fill_between(test_smoothed.index, forecast_conf_int_smoothed.iloc[:, 0], forecast_conf_int_smoothed.iloc[:, 1], color='pink', alpha=0.3)
    plt.xlim(x_limits)
    plt.ylim(-5, 30)
    plt.title(f'ARIMA Model on Smoothed (Moving Average) Data: Window Size {window_size}')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.legend()
    plt.show()
